In [1]:
from IPython.display import clear_output

! git clone https://github.com/awarebayes/RecNN
! pip install -r ./RecNN/requirements.txt
! pip install ./RecNN
! pip install gdown

clear_output()

In [2]:
! wget http://files.grouplens.org/datasets/movielens/ml-20m.zip
! gdown https://drive.google.com/uc?id=1EQ_zXBR3DKpmJR3jBgLvt-xoOvArGMsL
! unzip ml-20m.zip
clear_output()

In [3]:
import numpy as np
import pandas as pd
from IPython.display import clear_output
import recnn

import torch
import torch.nn as nn
from tqdm.auto import tqdm
import torch_optimizer as optim

C:\Users\varun\AppData\Local\Programs\Python\Python39\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
articles_dataset = pd.read_csv("H&M Fashion/articles.csv")

In [5]:
articles_dataset.head()

,article_id,product_code,prod_name,product_type_no,product_type_name,product_group_name,graphical_appearance_no,graphical_appearance_name,colour_group_code,colour_group_name,...,department_name,index_code,index_name,index_group_no,index_group_name,section_no,section_name,garment_group_no,garment_group_name,detail_desc
0,108775015,108775,Strap top,253,Vest top,Garment Upper body,1010016,Solid,9,Black,...,Jersey Basic,A,Ladieswear,1,Ladieswear,16,Womens Everyday Basics,1002,Jersey Basic,Jersey top with narrow shoulder straps.
1,108775044,108775,Strap top,253,Vest top,Garment Upper body,1010016,Solid,10,White,...,Jersey Basic,A,Ladieswear,1,Ladieswear,16,Womens Everyday Basics,1002,Jersey Basic,Jersey top with narrow shoulder straps.
2,108775051,108775,Strap top (1),253,Vest top,Garment Upper body,1010017,Stripe,11,Off White,...,Jersey Basic,A,Ladieswear,1,Ladieswear,16,Womens Everyday Basics,1002,Jersey Basic,Jersey top with narrow shoulder straps.
3,110065001,110065,OP T-shirt (Idro),306,Bra,Underwear,1010016,Solid,9,Black,...,Clean Lingerie,B,Lingeries/Tights,1,Ladieswear,61,Womens Lingerie,1017,"Under-, Nightwear","Microfibre T-shirt bra with underwired, moulde..."
4,110065002,110065,OP T-shirt (Idro),306,Bra,Underwear,1010016,Solid,10,White,...,Clean Lingerie,B,Lingeries/Tights,1,Ladieswear,61,Womens Lingerie,1017,"Under-, Nightwear","Microfibre T-shirt bra with underwired, moulde..."


In [6]:
customers_dataset = pd.read_csv("H&M Fashion/customers.csv")

In [7]:
customers_dataset.head()

,customer_id,FN,Active,club_member_status,fashion_news_frequency,age,postal_code
0,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,NaN,NaN,ACTIVE,NONE,49.0,52043ee2162cf5aa7ee79974281641c6f11a68d276429a...
1,0000423b00ade91418cceaf3b26c6af3dd342b51fd051e...,NaN,NaN,ACTIVE,NONE,25.0,2973abc54daa8a5f8ccfe9362140c63247c5eee03f1d93...
2,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,NaN,NaN,ACTIVE,NONE,24.0,64f17e6a330a85798e4998f62d0930d14db8db1c054af6...
3,00005ca1c9ed5f5146b52ac8639a40ca9d57aeff4d1bd2...,NaN,NaN,ACTIVE,NONE,54.0,5d36574f52495e81f019b680c843c443bd343d5ca5b1c2...
4,00006413d8573cd20ed7128e53b7b13819fe5cfc2d801f...,1.0,1.0,ACTIVE,Regularly,52.0,25fa5ddee9aac01b35208d01736e57942317d756b32ddd...


In [8]:
transactions_dataset = pd.read_csv("H&M Fashion/transactions_train.csv")

In [9]:
# Convert the date to datetime64
transactions_dataset['t_dat'] = pd.to_datetime(transactions_dataset['t_dat'], format='%Y-%m-%d')
  
# Filter data between two dates
transactions_dataset_filtered = transactions_dataset.loc[(transactions_dataset['t_dat'] >= '2020-09-01')]

In [10]:
transactions_dataset_filtered.reset_index().head()

,index,t_dat,customer_id,article_id,price,sales_channel_id
0,30990055,2020-09-01,0001d44dbe7f6c4b35200abdb052c77a87596fe1bdcc37...,777148006,0.013542,1
1,30990056,2020-09-01,0001d44dbe7f6c4b35200abdb052c77a87596fe1bdcc37...,835801001,0.018627,1
2,30990057,2020-09-01,0001d44dbe7f6c4b35200abdb052c77a87596fe1bdcc37...,923134005,0.012695,1
3,30990058,2020-09-01,0001d44dbe7f6c4b35200abdb052c77a87596fe1bdcc37...,865929003,0.016932,1
4,30990059,2020-09-01,0005ed68483efa39644c45185550a82dd09acb07622acb...,863646004,0.033881,1


In [11]:
merged = pd.merge(articles_dataset, transactions_dataset_filtered, on="article_id")

In [12]:
merged.groupby("article_id")['customer_id'].count().sort_values(ascending=False).head()

article_id
751471001    1982
909370001    1868
918522001    1702
924243001    1683
918292001    1682
Name: customer_id, dtype: int64

In [13]:
merged_simplified = merged[["article_id", "customer_id", "t_dat"]]

In [14]:
merged_simplified["article_id_2"] = merged_simplified["article_id"]
f = {'article_id': 'first', 'customer_id': 'first', 'article_id_2': 'count', 't_dat': 'first'}
merged_simplified = merged_simplified.groupby(['article_id', 'customer_id', 't_dat'], as_index=False).agg(f)
merged_simplified_new = merged_simplified.rename({'article_id_2': 'quantity', 't_dat': 'when'}, axis='columns')
merged_simplified_new.head()

C:\Users\varun\AppData\Local\Temp\ipykernel_39112\3132972307.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged_simplified["article_id_2"] = merged_simplified["article_id"]


,article_id,customer_id,quantity,when
0,108775044,5b846c003cefba83854b861bf97d5b02318f945fa70574...,2,2020-09-17
1,108775044,65cdeef43054eeade12fb0c774c4ac2e4bc051b5305253...,2,2020-09-13
2,108775044,8270dfed94715623caa7c4e191e81adff2501609f7f29b...,1,2020-09-20
3,108775044,a995fb9182c93e9c70f134b3bf493cce1b666950af280b...,4,2020-09-03
4,108775044,b6b0f9581b6ba1be0731f2a8415a1bbfee8b42558da261...,1,2020-09-13


In [15]:
import pandas as pd
import numpy as np
import datetime
import random
import time

def random_string_date():
  return datetime.datetime.strptime('{} {} {} {}'.format(random.randint(1, 366),
                                                         random.randint(0, 23),
                                                         random.randint(1, 59),
                                                          2020), '%j %H %M %Y').strftime("%m/%d/%Y, %H:%M:%S")

def string_time_to_unix(s):
  return int(time.mktime(datetime.datetime.strptime(s, "%m/%d/%Y, %H:%M:%S").timetuple()))

size = 100000
n_emb = len(merged_simplified_new) #1000
n_usr = 1000
# mydf = pd.DataFrame({'article_id': np.random.randint(0, n_emb, size=size),
#                      'customer_id': np.random.randint(1, n_usr, size=size),
#                      'bought': np.random.randint(0, 2, size=size),
#                      'when': [random_string_date() for i in range(size)]})
data_embeddings = dict([(i, torch.tensor(np.random.randn(128)).float()) for i in range(n_emb)])
# mydf.head()

In [16]:
! mkdir DatasetForRN
import pickle

merged_simplified_new.to_csv('DatasetForRN/Created.csv', index=False)
with open('DatasetForRN/Created.pickle', 'wb') as handle:
    pickle.dump(data_embeddings, handle)

A subdirectory or file DatasetForRN already exists.


In [19]:
def prepare_my_dataset(args_mut, kwargs):

    # get args
    frame_size = kwargs.get('frame_size')
    key_to_id = args_mut.base.key_to_id
    df = args_mut.df

    df['quantity'] = df['quantity'].apply(lambda a: a) #(a - 1) * (1 - a) + a)
#     df['when'] = df['when'].apply(string_time_to_unix)
    df['article_id'] = df['article_id'].apply(key_to_id.get)

    users = df[['customer_id', 'article_id']].groupby(['customer_id']).size()
    users = users[users > frame_size].sort_values(ascending=False).index

    # If using modin: pandas groupby is sync and doesnt affect performance
    # if pd.get_type() == "modin": df = df._to_pandas()  
    ratings = df.sort_values(by='when').set_index('customer_id').drop('when', axis=1).groupby('customer_id')

    # Groupby user
    user_dict = {}

    def app(x):
        userid = x.index[0]
        user_dict[userid] = {}
        user_dict[userid]['items'] = x['article_id'].values
        user_dict[userid]['ratings'] = x['quantity'].values

    ratings.apply(app)

    args_mut.user_dict = user_dict
    args_mut.users = users

    return args_mut, kwargs

In [20]:
frame_size = 10
batch_size = 25


dirs = recnn.data.env.DataPath(
    base="DatasetForRN/",
    embeddings="Created.pickle",
    ratings="Created.csv",
    cache="frame_env.pkl", # cache will generate after you run
    use_cache=True # generally you want to save env after it runs
)

env = recnn.data.env.FrameEnv(
    dirs,
    frame_size,
    batch_size, 
    prepare_dataset=prepare_my_dataset # <- ! pass YOUR function here
) 

In [21]:
# test function
def run_tests():
    batch = next(iter(env.test_dataloader))
    loss = ddpg.update(batch, learn=False)
    return loss
  
value_net  = recnn.nn.Critic(1290, 128, 256, 54e-2)
policy_net = recnn.nn.Actor(1290, 128, 256, 6e-1)

# cuda = torch.device('cuda')
ddpg = recnn.nn.DDPG(policy_net, value_net)
# ddpg = ddpg.to(cuda)
plotter = recnn.utils.Plotter(ddpg.loss_layout, [['value', 'policy']],)

In [ ]:
from IPython.display import clear_output
import matplotlib.pyplot as plt
%matplotlib inline

plot_every = 3
n_epochs = 2

def learn():
    for epoch in range(n_epochs):
        for batch in tqdm(env.train_dataloader):
            loss = ddpg.update(batch, learn=True)
            plotter.log_losses(loss)
            ddpg.step()
            if ddpg._step % plot_every == 0:
                clear_output(True)
                print('step', ddpg._step)
                test_loss = run_tests()
                plotter.log_losses(test_loss, test=True)
                plotter.plot_loss()
            if ddpg._step > 100:
                return
            
learn()

  0%|                                                                                          | 0/383 [00:00<?, ?it/s]

In [ ]:
recommendation = policy_net(state)
value = value_net(state, recommendation)
print(recommendation)
print(value)

In [2]:
!cd RecNN && streamlit run examples/streamlit_demo.py

^C
